The purpose of this module is to perform high-level data manipulation and uncertainty analysis. 

In [1]:
from Uncertainty import *
import sympy as sym

Suppose you need to find the uncertainty in the following equation for the heat transfer coefficient for a heat exchanger, expressed as a function of measured parameters in the lab:

$$
UA = \frac{\frac{[m_h \times C_{ph} \times (T_{hi}-T_{ho}) + m_c \times C_{pc} \times (T_{co}-T_{ci})]/2}{T_{hi}-T_{ci} - T_{ho} - T_{co}}}{ln\frac{T_{hi}-T_{ci}}{T_{hi} - T_{ci}}}
$$

In [2]:
UA, m_h, c_ph, T_h_i, T_h_o, m_c, c_pc, T_c_i, T_c_o = sym.symbols("UA m_h c_ph T_h_i T_h_o m_c c_pc T_c_i T_c_o")

expr1 = (m_h*c_ph*(T_h_i-T_h_o) + m_c*c_pc*(T_c_o - T_c_i))

expr2 = (2*(T_h_i-T_c_o-T_h_o+T_c_i))*1/(sym.ln((T_h_i-T_c_o)/(T_h_o-T_c_i)))

expr = expr1/expr2

expr

(c_pc*m_c*(-T_c_i + T_c_o) + c_ph*m_h*(T_h_i - T_h_o))*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o)

To calculate the propagated error from the individual terms, you can use the RSS method:
$$
\sqrt{{\sum}_{i}^{n} \left(\frac{\partial f_i}{\partial x_i} U_{x_i} \right)^2}
$$

Calculating by hand or hardcoding the uncertainty can be tedious and often leads to mistakes, especially when you have to do this task for multiple expressions of equal complexity. Using the get_RSS() function, it can be calculated in one line of code. 

In [3]:
get_RSS(expr)

sqrt(U_T_c_i**2*(-c_pc*m_c*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o) - 2*(c_pc*m_c*(-T_c_i + T_c_o) + c_ph*m_h*(T_h_i - T_h_o))*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o)**2 + (c_pc*m_c*(-T_c_i + T_c_o) + c_ph*m_h*(T_h_i - T_h_o))/((-T_c_i + T_h_o)*(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o)))**2 + U_T_c_o**2*(c_pc*m_c*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o) + 2*(c_pc*m_c*(-T_c_i + T_c_o) + c_ph*m_h*(T_h_i - T_h_o))*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o)**2 - (c_pc*m_c*(-T_c_i + T_c_o) + c_ph*m_h*(T_h_i - T_h_o))/((-T_c_o + T_h_i)*(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o)))**2 + U_T_h_i**2*(c_ph*m_h*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o) - 2*(c_pc*m_c*(-T_c_i + T_c_o) + c_ph*m_h*(T_h_i - T_h_o))*log((-T_c_o + T_h_i)/(-T_c_i + T_h_o))/(2*T_c_i - 2*T_c_o + 2*T_h_i - 2*T_h_o)**2 + (c_pc*m_c*(-T_c_i + T_c_o

Even more conveniently, you do not need to initialize the variables as sympy objects:

In [4]:
get_RSS("x/y^2")

sqrt(U_x**2/y**4 + 4*U_y**2*x**2/y**6)

You are also able to evaluate the RSS by just providing the values for each variable. For the uncertainties, just put $U_$ followed by the variable name. Say we want the uncertainty for the moment of inertia of a pendulum for a charpy impact test.

In [5]:
get_RSS("b*h^3 - (b-2*t)*(h-2*t)^3", b = .25, h = .5, t = .125, 
        U_b = .005, U_h = .005, U_t = .001, evaluate = True)
#the order you put in the keywords does not matter

0.00108579697831823

If you are missing a variable value the code helps you identify what is missing, say I forgot the value for $t$:

In [6]:
get_RSS("b*h^3 - (b-2*t)*(h-2*t)^3", b = .25, h = .5, 
        U_b = .005, U_h = .005, U_t = .001, evaluate = True)

ValueError: Symbol t value MUST be provided

Lastly, if you are working in Microsoft Excel and want to put the function directly into Excel, you can use the boolean excel = True (Just a minor convenience, really).

In [7]:
get_RSS("b*h^3 - (b-2*t)*(h-2*t)^3", b = .25, h = .5, 
        U_b = .005, U_h = .005, U_t = .001, excel = True)

'sqrt(U_b^2*(h^3 - (h - 2*t)^3)^2 + U_h^2*(3*b*h^2 - 3*(b - 2*t)*(h - 2*t)^2)^2 + U_t^2*(6*(b - 2*t)*(h - 2*t)^2 + 2*(h - 2*t)^3)^2)'

Another way to calculate propagated error is to use a Monte Carlo simulation, the function MonteCarlo() takes in keyword arguments similar to get_RSS():

In [8]:
MonteCarlo("b*h^3 - (b-2*t)*(h-2*t)^3", b = .25, h = .5, 
        U_b = .005, U_h = .005, U_t = .001, t = .125)

0.03125 +/- 0.001058668858564157


As you can see you get a similar value for uncertainty. The default number of simulations is $10,000$ but you can adjust it with the argument N = number of simulations. The code returns 2 standard deviations. 

Another function exists to get the uncertainty for numerical integration and calculating slope, which is convenient if you want to find the work done on a specimen $W$ in a charpy impact test or find the uncertainty in the modulus of elasticity $E$. 

In [4]:
x = [1,2,3,4]
U_x = [1, 1,1, 1]

y = [2,4,6,10]
U_y = [1, 1, 1, 1]

a = getMonteCarlo(x, y, U_x, U_y, slope = True)

a

(2.6000000000000005, 1.3353981112406787e-15)

In [10]:
x = [0,2,3,4,5]
U_x = [.18, .19, .21, .17, .17,]

y = [0,4,6,8,10]
U_y = [.21, .25, .24, .23, .23]

a = getMonteCarlo(x, y, U_x, U_y, slope = True)

a

(1.999881300567056, 0.005557350636297385)

These uncertainty calculators were created to be convenient and high-level on their own, but they are also used inside other functions for analyzing UTM data. The function determines on its own which column is load and which is from the extensometer. Output of $E$ is in $GPa$, other stresses are in $MPa$. 

In [11]:
values = getStressStrain("Specimen_RawData_1.csv", area = 41.93, mass = 67.01, volume = 8.824, MC_sim = True)

print(f"The modulus of elasticity is {values[2]:.2f} +/- {values[3]:.3f} GPa")

The modulus of elasticity is 185.21 +/- 1.265 GPa
